In [1]:
# compile structure for irs soi data
# https://www.irs.gov/statistics/soi-tax-stats-historic-table-2

import re
import requests
import requests_cache
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

requests_cache.install_cache()

result = requests.get('https://www.irs.gov/statistics/soi-tax-stats-historic-table-2')
assert result.status_code == 200
c = result.content

soup = BeautifulSoup(c)
state_links = soup.find('table').find_all('a')

state_refs = []

for a in state_links:
    state_dict = {}
    
    state_dict['name'] = a.string.strip()
    
    groups = re.search('\d\din(?P<id>\d\d)(?P<abbrev>\w\w)', a.attrs['href'])
    state_dict.update(groups.groupdict())
    
    state_refs.append(state_dict)
    
df_state_refs = pd.DataFrame(state_refs).sort_values('id').reset_index(drop=True)

display(df_state_refs)

,abbrev,id,name
0,al,01,Alabama
1,ak,02,Alaska
2,az,03,Arizona
3,ar,04,Arkansas
4,ca,05,California
5,co,06,Colorado
6,ct,07,Connecticut
7,de,08,Delaware
8,dc,09,District of Columbia
9,fl,10,Florida


In [2]:
# create empty dfs dict to store data we grab from the irs site
CACHED_DFS = {}

In [3]:
# set of functions that help us get soi_data from the web as a pandas dataframe
# use: get_soi_df

from requests import Session
from urllib.parse import urljoin

class LiveServerSession(Session):
    def __init__(self, prefix_url=None, *args, **kwargs):
        super(LiveServerSession, self).__init__(*args, **kwargs)
        self.prefix_url = prefix_url

    def request(self, method, url, *args, **kwargs):
        url = urljoin(self.prefix_url, url)
        return super(LiveServerSession, self).request(method, url, *args, **kwargs)

def lookup_state(key, val):
    df = df_state_refs
    condition = df[key] == val
    state_info = df[condition].to_dict(orient='records')[0]
    return state_info

def get_soi_data(lookup, year):
    state_info = lookup_state(*lookup)
    filename = "{}in{}{}".format(
        str(year)[2:],
        state_info['id'],
        state_info['abbrev']
    )
    pattern = r'{}\.(xlsx?|zip)'.format(filename)
    url = soup.find('a', href=re.compile(pattern)).attrs['href']
    baseUrl = 'https://www.irs.gov'
        
    try:
        with LiveServerSession(baseUrl) as s:
            r = s.get(url)
        assert r.status_code == 200
    except:
        raise
        
    return r

def get_soi_df(lookup, year):
    '''
    get cumulative data as df given lookup and year
    '''
    from io import BytesIO
    from zipfile import ZipFile
    import itertools
    
    r = get_soi_data(lookup, year)
    
    print(r.url)
    
    if ((lookup, year) in CACHED_DFS):
        return CACHED_DFS[(lookup, year)]
    
    pd_options = {
        'header': None
    }
    
    if '.xls' in r.url:
        with BytesIO(r.content) as fh:
            df = pd.read_excel(fh, **pd_options)
    
    elif '.zip' in r.url:
        with ZipFile(BytesIO(r.content)) as my_zipfile:
            for file in my_zipfile.namelist():
                with my_zipfile.open(file) as fh:
                    df = pd.read_excel(fh, **pd_options)
    
    CACHED_DFS[(lookup, year)] = df
    
    return df

In [10]:
# set of functions that help us format soi dataframe
# use: format_soi_df


def rename_index(x):
    try:
        return x.strip().upper()
    except:
        return x

def get_indices(df):
    from collections import defaultdict

    iterator = df.iterrows()
    indices = defaultdict(lambda: None)

    while not np.all([indices[k] for k in ['place', 'amt']]):
        idx, row = next(iterator)

        if row.str.contains('alabama', flags=re.IGNORECASE).any() and not indices['place']:
            indices['place'] = idx

        if row.apply(str).str.contains('100,?000').any() and not indices['amt']:
            indices['amt'] = idx

    return dict(indices)

def format_soi_df(df):
    import numpy as np

    my_df = df.copy()
        
    # identity target rows for indexing
    indices = get_indices(my_df)

    # fill null holes in columns for multiindexing
    my_df.loc[indices['place']].fillna(method='ffill', inplace=True)
    my_df.loc[indices['place']].fillna('', inplace=True)
    my_df.loc[indices['place']] = my_df.loc[indices['place']].str.strip()
    my_df.loc[indices['amt']].fillna('All returns', inplace=True)
    
    def format_column(x):
        if isinstance(x, float) and np.isnan(x):
            return x
        elif isinstance(x, str) and 'Under' in x:
            x = '0'
        elif isinstance(x, str) and 'Breakeven' in x:
            x = '0'
        elif isinstance(x, str) and 'All' in x:
            x = '-1'
        else:
            x = str(x)

        try:
            return float(x.split()[0].replace('$', '').replace(',', ''))
        except:
            return x.split()[0]

    my_df.loc[indices['amt']] = my_df.loc[indices['amt']].apply(format_column)

    # drop rows with any null values in arbitrary range – we don't need these anymore
    my_df = my_df[~my_df.iloc[:, 1:4].isnull().any(axis=1)]
    
    # get rid of footnotes in data
    my_df.replace('\s*\[\d+\]', '', regex=True, inplace=True)

    # set indices
    my_df = my_df.transpose().set_index([*indices.values()])
    my_df.index.set_names([*indices.keys()], inplace=True)
    my_df.rename(index=rename_index, level='place', inplace=True)

    # set columns
    my_df.columns = my_df.iloc[0]
    my_df = my_df.iloc[1:]
    my_df.columns.rename('Item', inplace=True)
    
    # adjust alternate names
    my_df.rename(columns={
        'Returns Count': 'Number of returns'
    }, inplace=True)
    
    my_df = my_df[my_df.iloc[:, 3] != my_df.columns[3]] # drop rows that match header

    return my_df

In [5]:
# realign_tables unstacks IRS SOI csvs that organize the excel sheet in different
# tables rather than just one wide one

def realign_tables(df):    
    # locate new stacked tables using the table heading
    new_starts = df[df.iloc[:, 0].str.contains('^Table 2.', flags=re.IGNORECASE, na=False)].index
    
    # exception for older tables
    if len(new_starts) == 0:
        new_starts = df[df.iloc[:, 0].str.contains('^Tax Year 200', flags=re.IGNORECASE, na=False)].index
    
    # drop the first entry; it'll always be 0
    new_starts = new_starts.drop(0)
    
    # if any entries besides the first...
    if new_starts.any():
        dfs = []
        iterator = iter(new_starts.values)
        
        curr = 0
                
        for next_val in new_starts.values:
            start = curr
            curr = next_val
            
            dfs.append(df.loc[start:(curr-1), :].reset_index(drop=True))
        
        dfs.append(df.loc[curr:, :].reset_index(drop=True))
        
        concat = pd.concat(dfs, axis=1, ignore_index=True)
        
        # begone null columns
        concat = concat.loc[:, ~concat.isnull().all()]
        
        return concat
        
    # ...else just return the data, no alignment needed    
    else:
        return df


In [6]:
def get_cleaned_cm_df(year, do_print=False):
    import datetime
    import os
    
    
    if do_print: print(datetime.datetime.now(), year, 'Fetching data')
    df = get_soi_df(('abbrev', 'cm'), year)
    if do_print: print(datetime.datetime.now(), year, 'Realigning data')
    df = realign_tables(df)
    if do_print: print(datetime.datetime.now(), year, 'Formatting data')
    
    if os.path.isfile(f'./working/{year}.csv'):
        df = pd.read_csv(f'./working/{year}.csv', index_col=0, header=0)
    
    my_df = format_soi_df(df)
    
    return my_df

In [34]:
import datetime

RANGES = {
    'Total': (0,),
    'Under $50k': (0, 5e4),     #    0 <= ... <  50k
    '$50-$100k': (5e4, 1e5),    #  50k <= ... < 100k
    '$100-$200k': (1e5, 2e5),   # 100k <= ... < 200k
    '$200k and greater': (2e5,) # 200k <= ...
}

DIMENSIONS = [
    'Number of returns',
#     'Adjusted gross income (AGI)',
]

def build_query(my_place, lte, gt=None):
    my_query = ''
    
    my_query += f'{lte} <= amt'
    
    if (gt):
        my_query += f' < {gt} '
    
    my_query += f' and place == "{my_place}"'
    
    return my_query

data = []

def format_place(x):
    df = df_state_refs[df_state_refs.name.str.contains(x, case=False)]
    return df.abbrev.iloc[0]

for year in range(2016, 1999, -1):
    my_df = get_cleaned_cm_df(year)
    my_place = "Illinois".upper()

    for range_name, my_range in RANGES.items():
        my_query = build_query(my_place, *my_range)
        
        for my_dimension in DIMENSIONS:
            my_value = my_df.query(my_query)[my_dimension].astype('int64').sum()

            data.append((
                my_dimension,
                f'1/1/{year}',
                format_place(my_place),
                range_name,
                my_value
            ))
        
df = pd.DataFrame(data)

display(df.head())

https://www.irs.gov/pub/irs-soi/16in54cm.xlsx
https://www.irs.gov/pub/irs-soi/15in54cm.xlsx
https://www.irs.gov/pub/irs-soi/14in54cm.xlsx
https://www.irs.gov/pub/irs-soi/13in54cm.xlsx
https://www.irs.gov/pub/irs-soi/12in54cm.zip
https://www.irs.gov/pub/irs-soi/11in54cm.zip
https://www.irs.gov/pub/irs-soi/10in54cm.xls
https://www.irs.gov/pub/irs-soi/09in54cm.xls
https://www.irs.gov/pub/irs-soi/08in54cm.xls
https://www.irs.gov/pub/irs-soi/07in54cm.xls
https://www.irs.gov/pub/irs-soi/06in54cm.xls
https://www.irs.gov/pub/irs-soi/05in54cm.xls
https://www.irs.gov/pub/irs-soi/04in54cm.xls
https://www.irs.gov/pub/irs-soi/03in54cm.xls
https://www.irs.gov/pub/irs-soi/02in54cm.xls
https://www.irs.gov/pub/irs-soi/01in54cm.xls
https://www.irs.gov/pub/irs-soi/00in54cm.xls


,0,1,2,3,4
0,Number of returns,1/1/2016,il,Total,6100100
1,Number of returns,1/1/2016,il,Under $50k,3569220
2,Number of returns,1/1/2016,il,$50-$100k,1397680
3,Number of returns,1/1/2016,il,$100-$200k,827510
4,Number of returns,1/1/2016,il,$200k and greater,305690


In [80]:
df.columns = ('dimension', 'date', 'group', 'range', 'val')

pivot = pd.pivot_table(df, values='val', columns='range', index=['dimension', 'group', 'date'])

pivot = pivot[['Under $50k', '$50-$100k', '$100-$200k', '$200k and greater']]

In [81]:
subset = pivot.xs('1/1/2007', level='date', drop_level=False)
subset[:] = ''
pivot.update(subset)
pivot

/Users/pjudge/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pjudge/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


range                            Under $50k $50-$100k $100-$200k  \
dimension         group date                                       
Number of returns il    1/1/2000    3980066   1253148     410470   
                        1/1/2001    3951561   1272520     420215   
                        1/1/2002    3922761   1271560     416318   
                        1/1/2003    3885585   1271678     433796   
                        1/1/2004    3840048   1299017     473357   
                        1/1/2005    3832554   1319905     513157   
                        1/1/2006    3855445   1357931     572474   
                        1/1/2007                                   
                        1/1/2008    3865517   1390269     649579   
                        1/1/2009    3849417   1345143     627582   
                        1/1/2010    3834120   1358185     651227   
                        1/1/2011    3860445   1358033     683830   
                        1/1/2012    3754050   1359260     717840   
                        1/1/2013    3721230   1370470     752490   
                        1/1/2014    3671840   1386000     790340   
                        1/1/2015    3638660   1399680     822510   
                        1/1/2016    3569220   1397680     827510   

range                            $200k and greater  
dimension         group date                        
Number of returns il    1/1/2000            143288  
                        1/1/2001            130819  
                        1/1/2002            125439  
                        1/1/2003            131696  
                        1/1/2004            150467  
                        1/1/2005            170577  
                        1/1/2006            193844  
                        1/1/2007                    
                        1/1/2008            207061  
                        1/1/2009            186041  
                        1/1/2010            200337  
                        1/1/2011            219720  
                        1/1/2012            245950  
                        1/1/2013            256510  
                        1/1/2014            282930  
                        1/1/2015            301130  
                        1/1/2016            305690

In [82]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/pjudge/.credentials/BGA Graphics-3edf4552f3a5.json', scope)

gc = gspread.authorize(credentials)

In [83]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe

worksheet = gc.open_by_key('1-Ayjw4mhGHuoO7Igo_3ecCf6OI-zjytvc3jNDZnnEd0').worksheet('data')

In [84]:
def blank_out_worksheet(worksheet):
    """
    totally blank out worksheet
    """
    from gspread_dataframe import get_as_dataframe, set_with_dataframe
    
    zeroed_df = get_as_dataframe(worksheet)
    
    # set vals to null
    zeroed_df[:] = np.nan
    
    # set cols to null
    zeroed_df.rename(columns=lambda x: np.nan, inplace=True)
    
    # set worksheet to blank dataframe
    set_with_dataframe(worksheet, zeroed_df)

In [93]:

new_df = pivot.reset_index().drop(['dimension', 'group'], 'columns')

new_df['$50k and greater'] = new_df.loc[:, '$50-$100k':].sum(axis=1)

new_df[['Under $50k', '$50k and greater']].sum(axis=1)

# new_df = new_df[['date', 'Under $50k', '$50k and greater']]

# blank_out_worksheet(worksheet)
# set_with_dataframe(worksheet, new_df)

0     5786972
1     5775115
2     5736078
3     5722755
4     5762889
5     5836193
6     5979694
7            
8     6112426
9     6008183
10    6043869
11    6122028
12    6077100
13    6100700
14    6131110
15    6161980
16    6100100
dtype: object